In [1]:
from sympy import *
from IPython.display import display
init_printing(use_latex='mathjax')

In [2]:
x, y, z = symbols('x y z')

### 1. 例子

In [3]:
eq1 = Eq(x + 2 * y + z, 2)
eq2 = Eq(3 * x + 8 * y + z, 12)
eq3 = Eq(4 * y + z, 2)
(eq1, eq2, eq3)

(x + 2⋅y + z = 2, 3⋅x + 8⋅y + z = 12, 4⋅y + z = 2)

In [4]:
A = Matrix([[1, 2, 1], [3, 8, 1], [0, 4, 1]])
A

⎡1  2  1⎤
⎢       ⎥
⎢3  8  1⎥
⎢       ⎥
⎣0  4  1⎦

In [5]:
X = Matrix([[x], [y], [z]])
X

⎡x⎤
⎢ ⎥
⎢y⎥
⎢ ⎥
⎣z⎦

In [32]:
b = Matrix([[2], [12], [2]])
b

⎡2 ⎤
⎢  ⎥
⎢12⎥
⎢  ⎥
⎣2 ⎦

### 2. 矩阵乘法规则

In [6]:
a1, a2, a3, a4 = symbols('a1 a2 a3 a4')
b1, b2, b3, b4 = symbols('b1 b2 b3 b4')

In [7]:
A = Matrix([[a1, a2], [a3, a4]])
A

⎡a₁  a₂⎤
⎢      ⎥
⎣a₃  a₄⎦

In [8]:
B = Matrix([[b1, b2], [b3, b4]])
B

⎡b₁  b₂⎤
⎢      ⎥
⎣b₃  b₄⎦

In [9]:
A * B

⎡a₁⋅b₁ + a₂⋅b₃  a₁⋅b₂ + a₂⋅b₄⎤
⎢                            ⎥
⎣a₃⋅b₁ + a₄⋅b₃  a₃⋅b₂ + a₄⋅b₄⎦

### .python实现

In [10]:
from pandas import DataFrame
from itertools import product
from more_itertools import chunked

In [11]:
A = DataFrame({0:['a1', 'a3'], 1:['a2', 'a4']})
A

,0,1
0,a1,a2
1,a3,a4


In [12]:
B = DataFrame({0:['b1', 'b3'], 1:['b2', 'b4']})
B

,0,1
0,b1,b2
1,b3,b4


In [13]:
rowListOfA =  map(lambda x: list(x[1]), A.iterrows())
rowListOfA

[['a1', 'a2'], ['a3', 'a4']]

In [14]:
colListOfB = map(lambda x: list(x[1]), B.iteritems())
colListOfB

[['b1', 'b3'], ['b2', 'b4']]

In [15]:
it = chunked(product(rowListOfA, colListOfB), len(rowListOfA))
for row in it:
    for a, b in row:
        print '%s*%s + %s*%s  ' %(a[0], b[0], a[1], b[1]),
    print

a1*b1 + a2*b3   a1*b2 + a2*b4  
a3*b1 + a4*b3   a3*b2 + a4*b4  


### 2. 消元的目标
得到一系列矩阵: E1, E2, E3, ..., En <br>
E1 \* E2 \* E3 \* ... \* En \* A为: <br>

In [50]:
n = 5
eye(n)

⎡1  0  0  0  0⎤
⎢             ⎥
⎢0  1  0  0  0⎥
⎢             ⎥
⎢0  0  1  0  0⎥
⎢             ⎥
⎢0  0  0  1  0⎥
⎢             ⎥
⎣0  0  0  0  1⎦

这种矩阵我们叫他"identity matrix", 中文还不知道怎么翻译

### 3. 消元过程

In [43]:
A = Matrix([[1, 2, 1], [3, 8, 1], [0, 4, 1]])
A

⎡1  2  1⎤
⎢       ⎥
⎢3  8  1⎥
⎢       ⎥
⎣0  4  1⎦

#### (1). E21思路
第1行保持不变: [1, 0, 0] <br>
第2行, 要消第1列, 使用第1行*-3 + 第2行: [-3, 1, 0] <br>
第3行保持不变: [0, 0, 1] <br>

In [17]:
E21 = Matrix([[1, 0, 0], [-3, 1, 0], [0, 0, 1]])
E21

⎡1   0  0⎤
⎢        ⎥
⎢-3  1  0⎥
⎢        ⎥
⎣0   0  1⎦

In [18]:
E21 * A

⎡1  2  1 ⎤
⎢        ⎥
⎢0  2  -2⎥
⎢        ⎥
⎣0  4  1 ⎦

#### (2). E32思路
第1行不变: [1, 0, 0] <br>
第2行不变: [0, 1, 0] <br>
第3行, 要消第2列, 第2行 * -2 + 第3行: [0, -2, 1]

In [19]:
E32 = Matrix([[1, 0, 0], [0, 1, 0], [0, -2, 1]])
E32

⎡1  0   0⎤
⎢        ⎥
⎢0  1   0⎥
⎢        ⎥
⎣0  -2  1⎦

In [20]:
U = E32 * (E21 * A)
U

⎡1  2  1 ⎤
⎢        ⎥
⎢0  2  -2⎥
⎢        ⎥
⎣0  0  5 ⎦

#### (3). E23思路
第1行不变: [1, 0, 0] <br>
第2行, 要消第3列, 第2行*5 + 第3行*2: [0, 5, 2] <br>
第3行不变: [0, 0, 1] <br>

In [44]:
E23 = Matrix([[1, 0, 0], [0, 5, 2], [0, 0, 1]])
E23

⎡1  0  0⎤
⎢       ⎥
⎢0  5  2⎥
⎢       ⎥
⎣0  0  1⎦

In [45]:
U = E23 * (E32 * (E21 * A))
U

⎡1  2   1⎤
⎢        ⎥
⎢0  10  0⎥
⎢        ⎥
⎣0  0   5⎦

### 4. 矩阵乘法性质
矩阵乘法没有交换律,但是有结合律 <br>
A \* (B \* C) = (A \* B) \* C
所以:

In [23]:
U = (E32 * E21) * A
U

⎡1  2  1 ⎤
⎢        ⎥
⎢0  2  -2⎥
⎢        ⎥
⎣0  0  5 ⎦

### 5. 逆矩阵

### (1)定义
如果: E**-1 * E = I <br>
我们就称E**-1 为 E的逆矩阵<br>

### (2). 逆矩阵求解方程原理
A \* X = b <br>
==> A \*\* -1 \* A \* X = A\*\*-1 \* b <br>
==> I \* X = A\*\*-1 \* b <br>
I 形如: <br>
[1, 0, 0] <br>
[0, 1, 0] <br>
[0, 0, 1] <br>
X 形如: [x, y, z] <br>
==> I \* X = X <br>
==> X = A \*\* -1 b <br>

### (3). 用Sympy求逆矩阵

In [61]:
_A = A.inv() 

In [62]:
_A * A

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

In [63]:
_A * b

⎡2 ⎤
⎢  ⎥
⎢1 ⎥
⎢  ⎥
⎣-2⎦